In [ ]:
##########
""" 
THESIS EXTENSION: MS-VECM 

with (EM) 



"""

In [ ]:
## VECM MODEL EST: 
https://towardsdatascience.com/long-run-relationships-between-fang-stocks-82baf183840c
https://www.researchgate.net/post/How_to_interpret_beta_and_alpha_in_VECM#:~:text=Indeed%2C%20in%20the%20VECM%2C%20causality,around%20the%20long%2Drun%20equilibrium.

## MARKOV LINKS: 
https://www.statsmodels.org/stable/examples/notebooks/generated/markov_regression.html
https://www.statsmodels.org/stable/generated/statsmodels.tsa.regime_switching.markov_regression.MarkovRegression.html
http://www.eviews.com/help/helpintro.html#page/content/switching-Background.html 
https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.spearmanr.html    
    
## AUTOREG
https://www.statsmodels.org/stable/examples/notebooks/generated/markov_autoregression.html

    
## EXPECTATION MAXIMISATION EM ALG
https://machinelearningmastery.com/expectation-maximization-em-algorithm/
https://au.mathworks.com/help/econ/specifying-lag-operator-polynomials-interactively.html    
    https://towardsdatascience.com/implement-expectation-maximization-em-algorithm-in-python-from-scratch-f1278d1b9137
    
    #in (R)
https://github.com/kjartako/MS_VAR/tree/master/MS_VECM
    
#johansen coint test:
https://www.statsmodels.org/stable/generated/statsmodels.tsa.vector_ar.vecm.JohansenTestResult.html#statsmodels.tsa.vector_ar.vecm.JohansenTestResult
    

In [1]:
#system packages

import sys
import warnings
import os 
import traceback #obs? 
if not sys.warnoptions:
    warnings.filterwarnings("once")  

In [2]:
#base packages:
import collections
import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats, integrate, optimize
import math
import datetime
from datetime import date, timedelta

## graphing packages ## 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

C:\Users\JasonSpano\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
#optional packages
from statsmodels.tsa.base.datetools import dates_from_str
from IPython.display import display
pd.options.display.max_columns = 60  #allow DF.head to show all columns in notebook
from see import see
from tabulate import tabulate 
#from io import StringIO

C:\Users\JasonSpano\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
#packages for the econometrics / models

##statsmodels 
import statsmodels.api as sm
from statsmodels.tsa.vector_ar import vecm
#import statsmodels.tsa.stattools as ts 
from statsmodels.tsa.stattools import adfuller, zivot_andrews
import statsmodels.formula.api as smf  #VAR package contained within 
import statsmodels.tsa.api as smt
from statsmodels.tsa.regime_switching import markov_autoregression

#archpackage
from arch.unitroot import DFGLS, ADF, KPSS, PhillipsPerron, ZivotAndrews, VarianceRatio
from arch.unitroot.cointegration import engle_granger, phillips_ouliaris

#import the functionality for detecting mathematical errors (E.G. types of linear algebra issues etc.)
from statsmodels.tools.sm_exceptions import ValueWarning
from arch.utility.exceptions import (
    InfeasibleTestException,
    InvalidLengthWarning,
    invalid_length_doc)
warnings.filterwarnings("once", category = ValueWarning)

C:\Users\JasonSpano\anaconda3\lib\site-packages\patsy\constraint.py:13: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  from collections import Mapping


In [5]:
import statsmodels.tsa.stattools as ts 

C:\Users\JasonSpano\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
#with ADF results: to be updated to VECM outcome (cointegration) eventually?
sorted_alldata_df_final_adf_results = pd.read_pickle("./sorted_alldata_df_final_adf_results.pkl")

In [ ]:
class color:
   #a simplified way to easily call different style elements for displaying outputs
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   GREEN = '\033[92m'
   HEADER = '\033[95m' 
   OKBLUE = '\033[94m'
   WARNING = '\033[93m'
   FAIL = '\033[91m'
   END = '\033[0m'

## MARKOV SWITCHING DYNAMIC REGRESSIONS
* Regime Switching in the price discovery leadership of CDS and/or bonds in different periods. 


####  Factors to consider in the models: 
* K_regimes: 
    * Number of possible regimes; there is some sound arguments to using 3 regimes (As in Guidolin, 2019). However, this paper considers only two possible regimes.
* Trend:
    * Once again, the trend variable will be set to an intercept (c)
* Switching_Variance:
    * Whether or not there will be regime-specific heteroskedasicity; specifically, whether the error term is allowed to have a switching variance
* Switching_Exog:
    * Allow all regression coefficients to switch across regimes

In [ ]:
## GRAPH TEST / VIEW CODE ## 
gvkey_selection = 1045 # 176404 #3226 #1440 #3226 #1078

gvkey_group_df = sorted_alldata_df_final_adf_results.loc[sorted_alldata_df_final_adf_results.gvkey==gvkey_selection]
print(" COMPANY == ", gvkey_group_df.company_name.iloc[0], '\n', "SECTOR ==  ", gvkey_group_df.sector.iloc[0])

fig, (ax1, ax2, ax3) = plt.subplots(3, figsize=(12,10))
fig.suptitle('Single GVKEY: ({}) CDS Spread Subplots'.format(gvkey_selection))

gvkey_group_df.groupby(['year_index'])['market_cds_spread'].plot(ax=ax1)
ax1.set_ylabel('CDS Spread')
ax1.axes.get_xaxis().set_visible(False)

## OG
gvkey_group_df.groupby(['year_index']).mean()['market_cds_spread'].plot(ax=ax2);

## to change to rolling annual mean: 
#gvkey_group_df['market_cds_spread'].rolling(252).mean().plot(ax=ax2);
ax2.set_ylabel('Annual Mean CDS Spread')
ax2.set_xlabel('Year')
ax2.axes.get_xaxis().set_visible(True)


gvkey_group_df.groupby(['year_index'])['pecds'].plot(ax=ax3)
ax3.set_ylabel('PECDS')
ax3.axes.get_xaxis().set_visible(False);

In [ ]:
##fulldata_df_weekly = fulldata_df.groupby('gvkey').resample('W-WED', on='trade_date').first()

In [ ]:
gvkey_selection = 1045 #3226 #1078 #1440

gvkey_frame = sorted_alldata_df_final_adf_results.loc[sorted_alldata_df_final_adf_results.gvkey==gvkey_selection]

cds = gvkey_frame['market_cds_spread']
pecds = gvkey_frame['pecds']

gvkey_frame_ms = pd.DataFrame(data=(cds, pecds)).T

test_dfgls_cds_trend_c = DFGLS(cds, trend='c', max_lags=10, method='BIC') 
### SWITCH TO VECM ORDER? 
print("DFGLS LAG ORDER: {}".format(test_dfgls_cds_trend_c.lags))

### CREATING LAGGED DEPENDENT VARIABLE: 
#exog_variables = pd.concat((cds.shift()[1:], pecds[1:]), axis=1)
#gvkey_frame_ms['market_cds_spread'][1:]

## ARRAY OF ONES FOR INTERCEPT ON TIME VARYING TRANSITION PROBABILITIES
#timevarying_transition_probs = np.ones_like(cds)[np.newaxis]
#timevarying_transition_probs = timevarying_transition_probs.T


#with switching intercept and non-lagged dependent variable

vecm_model_order = vecm.select_order(gvkey_frame_ms, maxlags=10)
print("Original Model Order: {}".format(vecm_model_order.bic))

vecm_model_order_bic = vecm_model_order.bic

if vecm_model_order_bic > 5:
    vecm_model_order_bic = 5

    
## OG VECM MOD:     
vecm_regression_model_og = vecm.VECM(gvkey_frame_ms, k_ar_diff=vecm_model_order_bic, 
                                  coint_rank=1, deterministic='co')

vecm_regresion_fit_og = vecm_regression_model_og.fit()

    
    
## MARKOV    
vecm_cointeg_rank_msvecm = vecm.select_coint_rank(gvkey_frame_ms, det_order=0, k_ar_diff = vecm_model_order_bic, 
                                                  method='trace', signif=0.05)

#ms_model_switching_var = sm.tsa.MarkovAutoregression(gvkey_frame_ms['market_cds_spread'][1:]

ms_model_switching_var = sm.tsa.MarkovRegression(gvkey_frame_ms['market_cds_spread'],
                                                 k_regimes=2, 
                                                 trend='c',
                                                 order=vecm_model_order_bic,
                                                 exog=gvkey_frame_ms['pecds'],
                                                 switching_trend=True,
                                                 switching_exog =True,
                                                 switching_variance=True)

ms_model_switching_var = ms_model_switching_var.fit()

print(ms_model_switching_var.summary())

In [ ]:
see(ms_model_switching_var)

In [ ]:
ms_model_switching_var.smoothed_marginal_probabilities

In [ ]:
## spearman's correlation: 
stats.spearmanr(ms_model_switching_var.smoothed_marginal_probabilities[1], 
               ms_model_switching_var.smoothed_marginal_probabilities[0])

In [ ]:
Spearman_Results = stats.spearmanr(ms_model_switching_var.smoothed_marginal_probabilities[1], 
               ms_model_switching_var.smoothed_marginal_probabilities[0])
Spearman_Results

In [ ]:
Spearman_Results[0] #[1] == pvalue

In [ ]:
ms_model_switching_var.expected_durations

In [ ]:
ms_model_switching_var.regime_transition

In [ ]:
## standard errorS: 
ms_model_switching_var.bse

In [ ]:
ms_model_switching_var.params

In [ ]:
ms_model_switching_var.smoothed_marginal_probabilities[1].plot()
#so [1] refers to the (high) CDS REGIME: 

## AUTOREGRESSIVE VERSION 



In [ ]:
gvkey_selection = 1045 #3226 #1078 #1440

gvkey_frame = sorted_alldata_df_final_adf_results.loc[sorted_alldata_df_final_adf_results.gvkey==gvkey_selection]

cds = gvkey_frame['market_cds_spread']
pecds = gvkey_frame['pecds']

gvkey_frame_ms = pd.DataFrame(data=(cds, pecds)).T

#exog_variables = pd.concat((cds.shift()[1:], pecds[1:]), axis=1)
#exog_variables = pd.DataFrame(data=(cds[:-1], pecds)).T

## ARRAY OF ONES FOR INTERCEPT ON TIME VARYING TRANSITION PROBABILITIES
#timevarying_transition_probs = np.ones_like(cds)[np.newaxis]
#timevarying_transition_probs = timevarying_transition_probs.T

#with switching intercept and non-lagged dependent variable

vecm_model_order = vecm.select_order(gvkey_frame_ms, maxlags=10)
print("Original Model Order: {}".format(vecm_model_order.bic))

vecm_model_order_bic = vecm_model_order.bic

if vecm_model_order_bic > 5:
    vecm_model_order_bic = 5


ms_model_switching_var = sm.tsa.MarkovAutoregression(gvkey_frame_ms['market_cds_spread'],
                                                 k_regimes=2, 
                                                 trend='c',
                                                 order= vecm_model_order_bic, #vecm_model_order.bic,
                                                 exog=gvkey_frame_ms['pecds'],
                                                 switching_ar=False,
                                                 switching_trend=True,
                                                 switching_exog =True,
                                                 switching_variance=True)

ms_model_switching_var = ms_model_switching_var.fit()

print(ms_model_switching_var.summary())

print('\n', "--"*25, '\n', "Correlation Param: ", '\n', 
stats.spearmanr(ms_model_switching_var.smoothed_marginal_probabilities[1], 
               ms_model_switching_var.smoothed_marginal_probabilities[0]))

## getting spearman correlation on groupby(DF)

In [ ]:
## long execution time! 

In [7]:
sorted_alldata_df_final_adf_results.groupby('gvkey').size()

C:\Users\JasonSpano\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


gvkey
1045       546
1078       620
1161      1885
1209      1471
1300      2374
          ... 
175263     265
176404    2092
176760    1760
179657     523
179700    1498
Length: 391, dtype: int64

In [8]:
key_testers = [104, 1078, 1161, 1209, 1300, 176760]

In [10]:
#sorted_alldata_df_final_adf_results.loc[sorted_alldata_df_final_adf_results.gvkey.isin(key_testers)]

C:\Users\JasonSpano\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
%%time

gvkey_list = []
sector_list = []

## MS-VECM ITEMS: 
smoothed_regime_probabilities = []
smoothed_regime_probabilities_pval = []
regime_durations = []
#smoothed_regime_prob_cds = []
#smoothed_regime_prob_bond = []


## group the data by GVKEY for the loop regressions: 
for i, val in enumerate(sorted_alldata_df_final_adf_results.loc[sorted_alldata_df_final_adf_results.gvkey.isin(key_testers)].groupby('gvkey')):
    gvkey = val[0]
    data = val[1]  
    gvkey_list.append(gvkey)             
    sector_string = data.sector
    sector_list.append(sector_string.iloc[0])
    
    
    ## ADD THIS: 
    # test_dfgls_cds_trend_c = DFGLS(cds, trend='c', max_lags=10, method='BIC') 
    
    
    #load the data to create the model for each GVKEY/LOOP
    cds = data.market_cds_spread
    pecds = data.pecds
    ms_vecm_data = pd.DataFrame(data=(cds, pecds)).T
    
    ## fit the model with maximum of 2 regimes
    ms_model_switching_var = sm.tsa.MarkovRegression(ms_vecm_data['market_cds_spread'],
                                                     k_regimes=2, 
                                                     trend='nc',
                                                     order=1, #vecm_model_order.bic, 
                                                     exog=ms_vecm_data['pecds'],
                                                     switching_trend=True,
                                                     switching_exog =True,
                                                     switching_variance=True)

    ms_model_switching_var = ms_model_switching_var.fit()
    
    
    ## calculate spearman correlation of the smoothed marginal(regime) probabilities: 
    spearman_corr = stats.spearmanr(ms_model_switching_var.smoothed_marginal_probabilities[0], 
               ms_model_switching_var.smoothed_marginal_probabilities[1])
    
    smoothed_regime_probabilities.append(np.round(spearman_corr[0],4))
    smoothed_regime_probabilities_pval.append(np.round(spearman_corr[1],4))
    
    regime_durations.append(np.round(ms_model_switching_var.expected_durations,4))
    
    
ms_vecm_results_df = pd.DataFrame(data=(gvkey_list, sector_list, 
                                       smoothed_regime_probabilities, smoothed_regime_probabilities_pval,
                                       regime_durations)).T

ms_vecm_results_df.columns = ['gvkey','sector','regime_corr','regime_corr_pval', 'regime_durations']


C:\Users\JasonSpano\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Users\JasonSpano\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'


Wall time: 6.93 s


In [16]:
ms_vecm_results_df

,gvkey,sector,regime_corr,regime_corr_pval,regime_durations
0,1078,Health Care,-0.9787,0,"[55.4033, 159.1551]"
1,1161,Technology,-0.992,0,"[319.0686, 844.9735]"
2,1209,Materials,-0.7774,0,"[351.9797, 1321.6516]"
3,1300,Capital Goods,-0.9914,0,"[185.8944, 236.7948]"
4,176760,Materials,-0.997,0,"[1008.8373, 2121.0133]"


In [18]:
ms_vecm_results_df

,gvkey,sector,regime_corr,regime_corr_pval,regime_durations
0,1078,Health Care,-0.7189,0,"[150.2833, 253.214]"
1,1161,Technology,-0.9911,0,"[262.307, 475.815]"
2,1209,Materials,-0.9976,0,"[578.9461, 1060.3733]"
3,1300,Capital Goods,-1,0,"[149.5823, 438.1522]"
4,176760,Materials,-0.9726,0,"[372.0346, 540.1661]"


In [17]:
%%time

gvkey_list = []
sector_list = []

## MS-VECM ITEMS: 
smoothed_regime_probabilities = []
smoothed_regime_probabilities_pval = []
regime_durations = []
#smoothed_regime_prob_cds = []
#smoothed_regime_prob_bond = []


## group the data by GVKEY for the loop regressions: 
for i, val in enumerate(sorted_alldata_df_final_adf_results.loc[sorted_alldata_df_final_adf_results.gvkey.isin(key_testers)].groupby('gvkey')):
    gvkey = val[0]
    data = val[1]  
    gvkey_list.append(gvkey)             
    sector_string = data.sector
    sector_list.append(sector_string.iloc[0])
    
    
    ## ADD THIS: 
    # test_dfgls_cds_trend_c = DFGLS(cds, trend='c', max_lags=10, method='BIC') 
    
    
    #load the data to create the model for each GVKEY/LOOP
    cds = data.market_cds_spread
    pecds = data.pecds
    ms_vecm_data = pd.DataFrame(data=(cds, pecds)).T
    
    ## fit the model with maximum of 2 regimes
    ms_model_switching_var = sm.tsa.MarkovRegression(ms_vecm_data['pecds'],
                                                     k_regimes=2, 
                                                     trend='nc',
                                                     order=1, #vecm_model_order.bic, 
                                                     exog=ms_vecm_data['market_cds_spread'],
                                                     switching_trend=True,
                                                     switching_exog =True,
                                                     switching_variance=True)

    ms_model_switching_var = ms_model_switching_var.fit()
    
    
    ## calculate spearman correlation of the smoothed marginal(regime) probabilities: 
    spearman_corr = stats.spearmanr(ms_model_switching_var.smoothed_marginal_probabilities[0], 
               ms_model_switching_var.smoothed_marginal_probabilities[1])
    
    smoothed_regime_probabilities.append(np.round(spearman_corr[0],4))
    smoothed_regime_probabilities_pval.append(np.round(spearman_corr[1],4))
    
    regime_durations.append(np.round(ms_model_switching_var.expected_durations,4))
    
    
ms_vecm_results_df = pd.DataFrame(data=(gvkey_list, sector_list, 
                                       smoothed_regime_probabilities, smoothed_regime_probabilities_pval,
                                       regime_durations)).T

ms_vecm_results_df.columns = ['gvkey','sector','regime_corr','regime_corr_pval', 'regime_durations']

C:\Users\JasonSpano\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Users\JasonSpano\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'


Wall time: 4.88 s


## SINGLE GVKEY TESTER ITEMS 

In [ ]:
omega = np.cov(vecm_regresion_fit.resid, rowvar=False)

np.linalg.cholesky(omega)

corrcoefs = np.corrcoef(vecm_regresion_fit.resid, rowvar=False)
corrcoefs

In [ ]:
#Original VECM base: CDS/PECDS


gvkey_selection = 4060

gvkey_group_df = sorted_alldata_df_final_adf_results.loc[sorted_alldata_df_final_adf_results.gvkey==gvkey_selection]
print(" COMPANY NAME IS: ", gvkey_group_df.company_name.iloc[0], '\n', "COMPANY SECTOR: ", gvkey_group_df.sector.iloc[0])

cds = gvkey_group_df['market_cds_spread']
pecds = gvkey_group_df['pecds']
vecm_data_base = pd.DataFrame(data=(cds, pecds)).T

vecm_model_base = vecm.select_order(vecm_data_base, maxlags=5)
optimal_lag_length_base = vecm_model_base.bic
    
vecm_cointeg_rank_base = vecm.select_coint_rank(vecm_data_base, det_order =1, k_ar_diff = optimal_lag_length_base, 
                                                  method='trace', signif=0.05)

#enforce statistical restriction that number of cointegrating relationships = (#variables - 1)
if vecm_cointeg_rank_base.rank >= 1:
    coint_rels_base = 1
else:
    print("insufficient number of cointegrating relationships for VECM model")
    
vecm_base = vecm.VECM(vecm_data_base, k_ar_diff=vecm_model_base.bic, 
                      coint_rank=coint_rels_base, deterministic='co')

vecm_base_res = vecm_base.fit()

print(vecm_base_res.summary())

print('\n\n', vecm_base_res.test_granger_causality(caused=0).summary(),'\n\n',
vecm_base_res.test_granger_causality(caused=1).summary())

In [ ]:
#from the loop: 
print(" COINT RANK: {}".format(gvkey_group_df.coint_rank.iloc[0]), '\n', "Valid Test Conditions: {}".format(
gvkey_group_df.ADF_RESULT.iloc[0]), '\n', "--"*25)


############
############ REMOVED BRACKETS FROM DENOMINATOR! 
gg_val = np.round(vecm_base_res.alpha[1,0] / vecm_base_res.alpha[1,0] - vecm_base_res.alpha[0,0], 2)

## BETA (wrong)
#gg_val = np.round(vecm_base_res.beta[1,0] / (vecm_base_res.beta[1,0] - vecm_base_res.beta[0,0]), 2)

sigma_u = vecm_base_res.sigma_u
chol_decomp = np.linalg.cholesky(vecm_base_res.sigma_u)

sig_1 = chol_decomp[0, 0]
sig_12 = chol_decomp[1, 0]
sig_2 = chol_decomp[1, 1]

## seem to be the inverse of the beta(s)??? 
loading_coef_1 = (vecm_base_res.beta[0] - vecm_base_res.const_coint[0,0] - vecm_base_res.alpha[0,0] - vecm_base_res.lin_trend_coint[0, 0])
loading_coef_2 = (vecm_base_res.beta[1] + vecm_base_res.const_coint[0,0] + vecm_base_res.alpha[1,0] + vecm_base_res.lin_trend_coint[0, 0])

x_1 = np.round(vecm_base_res.alpha[0,0],5)
x_2 = np.round(vecm_base_res.alpha[1,0], 5)

## BETA
#x_1 = vecm_base_res.beta[0,0]
#x_2 = vecm_base_res.beta[1,0]

HAS_1 = np.round(( x_2**2 * (sig_1**2 - sig_12**2 / sig_2**2) )  / ( x_2**2 * sig_1**2 - 2*x_1*x_2*sig_12 + x_1**2 * sig_2**2 ),4)
HAS_2 = np.round(( (x_2 * sig_1 - x_1 * sig_12 / sig_1)**2 ) / ( x_2**2 * sig_1**2 - 2*x_1*x_2*sig_12 + x_1**2 * sig_2**2 ),4)
HAS_mid = np.round((HAS_2 + HAS_1) / 2,4)

print(" GG VALUE {}".format(gg_val), '\n',
     "Loading Coef_1 {}".format(loading_coef_1), '\n',
     "Loading Coef_2 {}".format(loading_coef_2), '\n',
     "X_1: {} \n X_2: {}".format(x_1, x_2), '\n',
     "HAS_1: {} \n HAS_MID: {} \n HAS_2: {}".format(HAS_1, HAS_mid, HAS_2),'\n', "--"*25,
     '\n', "Sigma_U:", '\n', sigma_u, '\n', "--"*25, '\n', "Cholesky Decomposition of Sigma_U: ", '\n', chol_decomp, '\n',
       "--"*25, '\n',
     "Betas(1,0): {} \n Betas(0,0): {}".format(vecm_base_res.beta[1,0], vecm_base_res.beta[0,0]), '\n',  "--"*25, '\n',
     "Alphas(1,0): {} \n Alphas(0,0): {}".format(vecm_base_res.alpha[1,0], vecm_base_res.alpha[0,0]), '\n',  "--"*25)

In [ ]:
### GG  value close to (1) implies all variation is occuring in the CDS? 
### which is plausible as the bond is stationary? 

## check values of X_1 and X_2 --- 
## make sure the beta(1,0) or (0,1) are correctly represented in the formula for has / gg
## change names of has_1 / has_2 to upper/lower

## change DF format to be more succint (drop test_vals etc. wherever possible)

#### then, see how hard it would be to implement the markov model? 

In [ ]:
#insert GVKEY, view evolution of the key's CDS series & annual mean changes
gvkey_selection = 1919 #4060 #4060 #174130   #1045

gvkey_group_df = sorted_alldata_df_final_adf_results.loc[sorted_alldata_df_final_adf_results.gvkey==gvkey_selection]
print(" COMPANY NAME IS: ", gvkey_group_df.company_name.iloc[0], '\n', "COMPANY SECTOR: ", gvkey_group_df.sector.iloc[0])

#GVKEY's mean CDS spread
print(" GVKEY ({}) CDS MEAN IS: ".format(gvkey_selection),
     np.round(sorted_alldata_df_final_adf_results.loc[sorted_alldata_df_final_adf_results.gvkey==gvkey_selection]['market_cds_spread'].mean(),2))

print(" GVKEY ({}) CDS MAX VALUES IS: ".format(gvkey_selection),
     np.round(sorted_alldata_df_final_adf_results.loc[sorted_alldata_df_final_adf_results.gvkey==gvkey_selection]['market_cds_spread'].max(),2))

print(" GVKEY ({}) CDS MIN VALUES IS: ".format(gvkey_selection),
     np.round(sorted_alldata_df_final_adf_results.loc[sorted_alldata_df_final_adf_results.gvkey==gvkey_selection]['market_cds_spread'].min(),2))

#GVKEY's CDS spread standard deviation
print(" GVKEY ({}) CDS STANDARD DEVIATION IS: ".format(gvkey_selection),
     np.round(sorted_alldata_df_final_adf_results.loc[sorted_alldata_df_final_adf_results.gvkey==gvkey_selection]['market_cds_spread'].std(),2))

#GVKEY's CDS spread normalised standard deviation:
print(" GVKEY ({}) CDS NORMALISED STANDARD DEVIATION IS: ".format(gvkey_selection),
     np.round((sorted_alldata_df_final_adf_results.loc[sorted_alldata_df_final_adf_results.gvkey==gvkey_selection]['market_cds_spread'].std() / 
               sorted_alldata_df_final_adf_results.loc[sorted_alldata_df_final_adf_results.gvkey==gvkey_selection]['market_cds_spread'].mean()),2))

## CONSTRUCT THE PLOTS: 
fig, (ax1, ax2, ax3) = plt.subplots(3, figsize=(12,10))
fig.suptitle('Single GVKEY: ({}) CDS Spread Subplots'.format(gvkey_selection))

gvkey_group_df.groupby(['year_index'])['market_cds_spread'].plot(ax=ax1)
ax1.set_ylabel('CDS Spread')
ax1.axes.get_xaxis().set_visible(False)


gvkey_group_df.groupby(['year_index']).mean()['market_cds_spread'].plot(ax=ax2);
ax2.set_ylabel('Annual Mean CDS Spread')
ax2.set_xlabel('Year')

gvkey_group_df.groupby(['year_index'])['pecds'].plot(ax=ax3)
ax3.set_ylabel('PECDS')
ax3.axes.get_xaxis().set_visible(False);
